# **MLOps: Weights and Biases with Keras**

### Dhruv Pithadia



<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{intro-colab-keras} -->

In [1]:
!pip install wandb -qU
!pip install tensorflow
!pip install keras

In [2]:
# Log in to your W&B account
import wandb

wandb.login(key='54fd78e485fb18f6e4a2a748b75dfa4b178b8ccd')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/dhruvpithadia/.netrc


True

## 👟 **Run an experiment**
1️⃣. **Start a new run** and pass in hyperparameters to track

2️⃣. **Log metrics** from training or evaluation

3️⃣. **Visualize results** in the dashboard

# 🥕 **Simple Keras Classifier**
Run this model to train a simple MNIST classifier, and click on the project page link to see your results stream in live to a W&B project.

In [3]:
import random
import numpy as np
import tensorflow as tf
from wandb.integration.keras import WandbMetricsLogger,WandbModelCheckpoint
#from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# launch 2 experiments, trying different dropout rates
for run in range(2):
    # start a run, tracking hyperparameters
    wandb.init(
        project="wandb-keras-mnist",
        config={
            "layer_1": 512,
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "layer_2": 10,
            "activation_2": "softmax",
            "optimizer": "sgd",
            "loss": "sparse_categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 6,
            "batch_size": 256,
        },
    )
    config = wandb.config

    # get the data
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train, x_test = x_train / 255.0, x_test / 255.0

    # subset data for a faster demo
    x_train, y_train = x_train[::5], y_train[::5]

    x_test, y_test = x_test[::20], y_test[::20]

    labels = [str(digit) for digit in range(np.max(y_train) + 1)]

    # build the model
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
            tf.keras.layers.Dropout(config.dropout),
            tf.keras.layers.Dense(config.layer_2, activation=config.activation_2),
        ]
    )

    model.compile(optimizer=config.optimizer, loss=config.loss, metrics=[config.metric])

    # add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints
    wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}.keras"),
    ]

    model.fit(
        x=x_train,
        y=y_train,
        epochs=config.epoch,
        batch_size=config.batch_size,
        validation_data=(x_test, y_test),
        callbacks=wandb_callbacks,
    )

    # mark the run as finished
    wandb.finish()

wandb: Currently logged in as: eclipsetheassistant (eclipsetheassistant-svkm-s-narsee-monjee-institute-of-ma). Use `wandb login --relogin` to force relogin


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/6
 1/47 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.1094 - loss: 2.4518

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1481 - loss: 2.3391 - val_accuracy: 0.5660 - val_loss: 1.8897
Epoch 2/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.3988 - loss: 1.8830 - val_accuracy: 0.6880 - val_loss: 1.5498
Epoch 3/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5746 - loss: 1.5631 - val_accuracy: 0.7500 - val_loss: 1.2995
Epoch 4/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.6427 - loss: 1.3430 - val_accuracy: 0.7800 - val_loss: 1.1173
Epoch 5/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6910 - loss: 1.1801 - val_accuracy: 0.7920 - val_loss: 0.9842
Epoch 6/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.7193 - loss: 1.0618 - val_accuracy: 0.8080 - val_loss: 0.8846


epoch/accuracy,▁▄▆▇██
epoch/epoch,▁▂▄▅▇█
epoch/learning_rate,▁▁▁▁▁▁
epoch/loss,█▆▄▃▂▁
epoch/val_accuracy,▁▅▆▇██
epoch/val_loss,█▆▄▃▂▁
epoch/accuracy,0.72458
epoch/epoch,5
epoch/learning_rate,0.01
epoch/loss,1.03786
epoch/val_accuracy,0.808


Epoch 1/6
25/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1459 - loss: 2.3164  

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1969 - loss: 2.2484 - val_accuracy: 0.5980 - val_loss: 1.8624
Epoch 2/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 449ms/step - accuracy: 0.5608 - loss: 1.7994 - val_accuracy: 0.7300 - val_loss: 1.5148
Epoch 3/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7025 - loss: 1.4776 - val_accuracy: 0.7680 - val_loss: 1.2519
Epoch 4/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 508ms/step - accuracy: 0.7441 - loss: 1.2383 - val_accuracy: 0.7940 - val_loss: 1.0631
Epoch 5/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7736 - loss: 1.0692 - val_accuracy: 0.8100 - val_loss: 0.9290
Epoch 6/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 851ms/step - accuracy: 0.7915 - loss: 0.9464 - val_accuracy: 0.8160 - val_loss: 0.8307


epoch/accuracy,▁▅▇▇██
epoch/epoch,▁▂▄▅▇█
epoch/learning_rate,▁▁▁▁▁▁
epoch/loss,█▆▄▃▂▁
epoch/val_accuracy,▁▅▆▇██
epoch/val_loss,█▆▄▃▂▁
epoch/accuracy,0.79733
epoch/epoch,5
epoch/learning_rate,0.01
epoch/loss,0.91555
epoch/val_accuracy,0.816


You have now trained your first model using wandb! 👆 Click on the wandb link above to see your metrics.